<a href="https://colab.research.google.com/github/nzamski/WaterRecognition/blob/master/GPU_Run_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as f

from tqdm import tqdm
from PIL import Image
from glob import glob
from cv2 import imread
from pathlib import Path
from random import shuffle, seed
from datetime import datetime
from sklearn.metrics import accuracy_score, f1_score

### Experiments

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!nvidia-smi

Sun Aug 22 09:57:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    32W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
os.getcwd()

'/content'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Algorithm Run Tests

In [ ]:
class Hidden1(nn.Module):
    # define the model
    def __init__(self, length, hidden_size, activation):
        super().__init__()
        self.activation = activation
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(length * length * 3, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 2)

    # set activation functions for the layers
    def forward(self, x):
        x = self.flat(x)
        x = self.activation(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# set a fixed seed to shuffle paths by
seed(42)


def get_train_test_paths(test_ratio: float = 0.2):
    # extract the data from the dataset folder
    files = [file_name for file_name in Path(os.getcwd()+os.sep+'Water Bodies Dataset'+os.sep+'Images').rglob("*.jpg")]
    # randomize the order of the data
    random.shuffle(files)
    # separate test and train files
    first_train = int(test_ratio * len(files))
    test_path = files[:first_train]
    train_path = files[first_train:]
    return train_path, test_path


def load_image(file_name):
    # get image path and return as array
    img = Image.open(file_name)
    img.load()
    data = np.asarray(img, dtype="int32")
    return data


def get_mask_path(file_path):
    # gets source image path, returns mask path
    file_path = str(file_path).replace('Images', 'Masks')
    return file_path


def load_y(file_path):
    mask_path = get_mask_path(file_path)
    raw_image = imread(mask_path, 0)
    # convert pixel colors to absolute black & white
    binary_array = (raw_image < 128).astype(int)
    return binary_array


class FileLoader:
    def __init__(self, path, length: int, load_both: bool = True):
        self.length = length
        self.load_both = load_both
        # source image variables initialization
        self.source_image = load_image(path)
        self.source_current_x, self.source_current_y = -1, 0  # -1 is a temp starting point
        self.source_max_x = self.source_image.shape[0] - self.length
        self.source_max_y = self.source_image.shape[1] - self.length
        # mask image variables initialization
        self.mask_image = load_y(path) if load_both else None
        self.mask_current_x = int((length - 1) / 2) - 1 if load_both else None  # -1 is a temp starting point
        self.mask_current_y = int((length - 1) / 2) if load_both else None
        self.mask_max_x = self.mask_image.shape[0] - int((length - 1) / 2) if load_both else None
        self.mask_max_y = self.mask_image.shape[1] - int((length - 1) / 2) if load_both else None

    def advance_source_index(self):
        if self.source_current_x < self.source_max_x - self.length:
            # the slice doesn't exceed the columns
            self.source_current_x += 1
        else:
            # go to the next row and reset the x
            self.source_current_x = 0
            if self.source_current_y < self.source_max_y - self.length:
                self.source_current_y += 1
            else:
                # finished going through image
                return None
        return self.source_current_x, self.source_current_y

    def advance_mask_index(self):
        if self.mask_current_x < self.mask_max_x:  # no need in self.length, right?
            # the tag doesn't exceed the columns
            self.mask_current_x += 1
        else:
            # go to the next row and reset the x
            self.mask_current_x = 0
            if self.mask_current_y < self.mask_max_y:  # same
                self.mask_current_y += 1
            else:
                # finished going through image
                return None
        return self.mask_current_x, self.mask_current_y

    def get_next(self):
        # returns the next slice (+ tag)
        source_indices = self.advance_source_index()
        if source_indices is None:
            return None
        else:
            source_x, source_y = source_indices
            source_slice = self.source_image[
                           source_x:source_x + self.length,
                           source_y:source_y + self.length,
                           :]
        if self.load_both:
            mask_x, mask_y = self.advance_mask_index()
            mask_tag = self.mask_image[mask_x, mask_y]
            return source_slice, mask_tag
        return source_slice


class DataLoader:
    def __init__(self, path_list, length, batch_size: int = 4, load_both: bool = True, return_file_name: bool = False):
        self.path_list = path_list
        shuffle(self.path_list)
        self.length = length
        self.current_file_index = 0
        # expected batch size smaller than number of paths
        self.batch_size = batch_size
        self.load_both = load_both
        self.return_file_name = return_file_name
        self.active_file_loaders, self.file_names = self.setup_active_file_loaders()
        self.file_index = batch_size
        self.max_file_index = len(self.path_list)

    def __iter__(self):
        return self

    def setup_active_file_loaders(self):
        active_file_loaders, file_names = list(), list()
        for _ in tqdm(range(self.batch_size)):
            new_loader, file_name = self.get_next_loader()
            active_file_loaders.append(new_loader)
            file_names.append(file_name)
        return active_file_loaders, file_names

    def get_next_loader(self):
        # return the next file loader (by order) and its path
        if self.current_file_index < len(self.path_list):
            path = self.path_list[self.current_file_index]
            new_loader = FileLoader(path, self.length, self.load_both)
            self.current_file_index += 1
            return new_loader, path
        return None, None

    def __next__(self):
        # initiate lists to store a batch of data
        x_batch, y_batch = list(), list()
        for loader_index, data_loader in enumerate(self.active_file_loaders):
            # get next potential data loader
            loaded_result = data_loader.get_next()
            # end of current loader
            if loaded_result is None:
                new_loader, file_name = self.get_next_loader()
                # finished going through path list
                if new_loader is None:
                    del(self.active_file_loaders[loader_index])
                    del(self.file_names[loader_index])
                    # end of all loaders in active file loaders
                    if len(self.active_file_loaders) == 0:
                        raise StopIteration()
                # didn't finished going through all the data yet
                else:
                    data_loader = new_loader
                    self.active_file_loaders[loader_index] = data_loader
                    self.file_names[loader_index] = file_name
                    loaded_result = data_loader.get_next()
            # finished a batch
            if loaded_result is not None:
                x_batch.append(loaded_result[0])
                if self.load_both:
                    y_batch.append(loaded_result[1])
        # return all x, y batches
        results = [x_batch]
        if self.load_both:
            results.append(y_batch)
        if self.return_file_name:
            results.append(self.file_names)
        return results

In [ ]:
def get_train_test_paths(test_ratio: float = 0.2):
    # extract the data from the dataset folder
    files = [file_name for file_name in Path(os.getcwd()+os.sep+'drive'+os.sep+'MyDrive'+os.sep+'Water Bodies Dataset'+os.sep+'Images').rglob("*.jpg")]
    # randomize the order of the data
    shuffle(files)
    # separate test and train files
    first_train = int(test_ratio * len(files))
    test_path = files[:first_train]
    train_path = files[first_train:]
    return train_path, test_path


def get_mask_path(file_path):
    # disassemble and assemble data path to return mask path
    wdr = os.getcwd()+os.sep+'drive'+os.sep+'MyDrive'+os.sep+'Water Bodies Dataset'+os.sep+'Masks'
    file_path = str(file_path).split(os.sep)[-1]
    mask_path = wdr + os.sep + file_path
    return mask_path


def load_image(file_name):
    # get image path and return as array
    img = Image.open(file_name)
    img.load()
    data = np.asarray(img, dtype="int32")
    return data


def split_to_squares(image_path, length):
    # get image array from the path
    rgb_array = load_image(image_path)
    # store image height and width (in pixels)
    max_x, max_y, _ = rgb_array.shape
    # initiate list for image slices
    slices = []
    # move along the image and save every square to the list
    for corner_x in range(max_x - length + 1):  # why not +2 ?
        for corner_y in range(max_y - length + 1):
            # append the squared matrix to the list
            sub = rgb_array[corner_x:corner_x+length, corner_y:corner_y+length, :]
            slices.append(sub)
    return slices


def get_y(image_path, length):  # expected odd length
    # get mask from path
    binary_array = imread(image_path, 0)
    # store mask height and width (in pixels)
    max_x, max_y = binary_array.shape
    # convert pixel colors to absolute black & white
    binary_array = (binary_array < 128).astype(int)
    # initiate list for mask slices
    tags = []
    # move along the mask and save every square to the list
    for x in range(int((length - 1) / 2), max_x - int((length - 1) / 2)):
        for y in range(int((length - 1) / 2), max_y - int((length - 1) / 2)):
            # append the pixel to the list
            tag = binary_array[x, y]
            tags.append(tag)
    return tags


def get_x_y(file_path, length):
    X = split_to_squares(file_path, length)
    mask_path = get_mask_path(file_path)
    y = get_y(mask_path, length)
    return X, y


def fit_model(model, model_parameters, loss_function, batch_size, optimizer, input_image_length):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # retrieve train and test files
    train, test = get_train_test_paths()
    train_loader = DataLoader(train, input_image_length, batch_size)
    test_loader = DataLoader(test, input_image_length, batch_size)
    # initiate a list for loss accumulation
    losses = list()
    # assign the model
    model = model(*model_parameters).to(device)
    # set a loss function
    criterion = loss_function()
    # set an optimizer
    optimizer = optimizer(model.parameters(), lr=0.001)
    model.train()
    for epoch in range(10):
      epoch_start = datetime.now()
      epoch_loss = 0
      # iterate through all data pairs
      for x, y in tqdm(train_loader):
        # convert input pixel to tensor and flatten
        x = torch.tensor(x).float().to(device)
        # convert target to tensor
        tag = torch.tensor(y, dtype=torch.long).view(-1).to(device)
        # set all gradients to to zero
        optimizer.zero_grad()
        prediction = model(x)
        # activate cross entropy, calculate loss
        loss = criterion(prediction, tag)
        # back propagation
        loss.backward()
        optimizer.step()
        # update into current loss
        epoch_loss += loss.item()
      epoch_end = datetime.now()
      epoch_seconds = (epoch_end - epoch_start).total_seconds()
      model.eval()
      predicted, real = list(), list()
      for x, y in test_loader:
        real += y
        probabilities = model(x)
        _, batch_predicted = torch.max(probabilities, 1)
        batch_predicted = list(batch_predicted.view(-1))
        predicted += batch_predicted
      accuracy = accuracy_score(real, predicted)
      f1 = f1_score(real, predicted)
    

      df = pd.DataFrame({'Model Name':['Hidden1'],
                         'Iteration':[epoch],
                         'Hyperparameters':[f'''"input_image_length": 5
                         "hidden_layer_size": 10
                         "activation": "f.relu"
                         "optimizer": "optim.Adam"
                         "loss_function": "nn.CrossEntropyLoss"'''],
                         'Loss':[running_loss],
                         'Accuracy':[accuracy],
                         'F1':[f1],
                         'Iteration Training Seconds':[epoch_seconds]})
      df.to_csv('drive/MyDrive/water_bodies_results.csv', index=False, mode='a', header=False)
      print(df)

In [ ]:
model = Hidden1
batch_size = 256
input_image_length = 5
hidden_layer_size = 10
activation = f.relu
model_parameters = (input_image_length, hidden_layer_size, activation)
optimizer = optim.Adam
loss_function = nn.CrossEntropyLoss
fit_model(model, model_parameters, loss_function, batch_size, optimizer, input_image_length)

100%|██████████| 256/256 [00:05<00:00, 46.92it/s]
42721it [10:04, 70.68it/s]


KeyboardInterrupt: ignored